In [ ]:
import cv2
import numpy as np
import os

img_base_path = ""
img_folder_list = os.listdir(img_base_path)

for img_folder in img_folder_list:
    img_list = os.listdir(os.path.join(img_base_path,img_folder))
    for img in img_list:
        full_path= os.path.join(img_base_path,img_folder,img)
        print(full_path)
        name,_,model_num,condition, img_num = img.split('_')
        img_num = img_num.split('.')[0]
        print(model_num, img_num)
        break
    break

In [ ]:
# 신발 이미지 2진화

import cv2
import numpy as np
import os

img_base_path = "./원본이미지"
save_base_path = './mask'
img_folder_list = os.listdir(img_base_path)

for img_folder in img_folder_list:
    img_list = os.listdir(os.path.join(img_base_path,img_folder))
    print(img_list)
    for img in img_list:
        full_path= os.path.join(img_base_path,img_folder,img)
        
        print(full_path)
    
        full_path= os.path.join(img_base_path,img_folder,img)
        img_array = np.fromfile(full_path, np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        if image is None:
            raise ValueError("이미지를 찾을 수 없습니다. 파일 경로를 확인하세요.")

        if image.shape[2] == 4:
            alpha = image[:, :, 3]
            # 3) 0보다 크면 255, 아니면 0
            _, mask = cv2.threshold(alpha, 0, 255, cv2.THRESH_BINARY)
        else:
            # 알파 채널이 없는 경우 (검정 배경이라면)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # 검정(0) vs 비검정 픽셀 구분
            _, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

        # 4) 결과 저장
        # cv2.imshow("output_mask_path", mask)
        # cv2.imshow("Original", image)

        cv2.imwrite(os.path.join(save_base_path,img_folder,img), mask)
        # cv2.imwrite("bin_adaptive.png", binary_adapt)

        cv2.waitKey(0)
        cv2.destroyAllWindows()
    


In [ ]:
for img_folder in img_folder_list:
    img_list = os.listdir(os.path.join(img_base_path,img_folder))
    print(img_folder)        

In [ ]:
# 빨간색 원 == 이진화
import cv2
import numpy as np
import os
import pandas as pd

img_base_path = "./빈도수_image"
shoe_base_path = './mask'
save_base_path = './엑셀'
save_base_img_path = './결과이미지'

img_folder_list = os.listdir(img_base_path)

for img_folder in img_folder_list:
    img_list = os.listdir(os.path.join(img_base_path,img_folder))
    print(img_folder)
    name,_,model_num,_ = img_folder.split('_')
    if model_num == '1':
        data = {
            '이미지 번호': [7, 8, 9, 10, 1, 2, 3, 4, 5, 6],
            '번호':        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            '비율': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            '픽셀수': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        }
    elif model_num == '2':
        data = {
            '이미지 번호': [4, 5, 6, 7, 8, 9, 10, 1, 2, 3],
            '번호':        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            '비율': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            '픽셀수': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        }
    elif model_num == '3':
        data = {
            '이미지 번호': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            '번호':        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            '비율': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            '픽셀수': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        }
    df = pd.DataFrame(data)

    for img in img_list:
        full_path= os.path.join(img_base_path,img_folder,img)
        img_array = np.fromfile(full_path, np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        
        if image is None:
            raise ValueError("이미지를 찾을 수 없습니다. 파일 경로를 확인하세요.")
        
        height, width, _ = image.shape
        if height != 4000 and width != 2250:
            top_crop = 260
            bottom_crop = 260
            cropped_img = image[top_crop:height - bottom_crop, :]

            # 리사이즈 (4000x2250)
            image = cv2.resize(cropped_img, (4000, 2250), interpolation=cv2.INTER_LANCZOS4)
        
        name,_,model_num,condition, img_num = img.split('_')
        shoe_mask_image = cv2.imread(os.path.join(shoe_base_path,model_num,img_num), cv2.IMREAD_GRAYSCALE)
        
        img_num = img_num.split('.')[0]
        
        
        
        # 2. BGR 이미지를 HSV 색공간으로 변환
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # 2. 빨간색 영역 마스크 생성 (두 범위를 결합)
        lower_red1 = np.array([0, 255, 255])
        upper_red1 = np.array([20, 255, 255])
        mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)

        lower_red2 = np.array([160, 255, 255])
        upper_red2 = np.array([180, 255, 255])
        mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)

        red_mask = cv2.bitwise_or(mask1, mask2)
        # --- 노이즈 제거 ---
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        
        # cv2.imwrite('mask1.png',red_mask)
        # red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel, iterations=2)
        # cv2.imwrite('mask2.png',red_mask)
        # red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE, kernel, iterations=1)

        # --- 컨투어 검출 & 타원 채우기 ---
        contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filled = np.zeros_like(red_mask)  # 타원만 채울 빈 마스크
        for cnt in contours:
            if len(cnt) < 5 or cv2.contourArea(cnt) < 200:
                continue
            ellipse = cv2.fitEllipse(cnt)
            # thickness=-1 로 내부를 채움
            cv2.ellipse(filled, ellipse, 255, thickness=-1)

        shoe_pixel_count = cv2.countNonZero(shoe_mask_image)

        # 3) AND 연산으로 겹치는 부분 추출
        intersection = cv2.bitwise_and(shoe_mask_image, filled)

        # 4) 흰색 픽셀(겹치는 영역) 개수 계산
        overlap_count = cv2.countNonZero(intersection)
        print(f'겹치는 흰색 픽셀 개수: {overlap_count:,}')

        df.loc[df['번호'] == int(img_num), '픽셀수'] = overlap_count
        df.loc[df['번호'] == int(img_num), '비율'] = round((overlap_count/shoe_pixel_count)*100,2)
        # 결과 출력
        # cv2.imshow("Original Image", image)
        # cv2.imshow("Red Mask", red_mask)
        # cv2.imshow("Filled Ellipses", filled)
        # cv2.imshow("Shoe Mask", shoe_mask_image)
        # cv2.imshow('overlap_mask.png', intersection)
        # 결과저장

        cv2.imwrite(os.path.join(save_base_path,model_num,(name+'_'+'_'+img_num+"Original Image.png")), image)
        cv2.imwrite(os.path.join(save_base_path,model_num,(name+'_'+'_'+img_num+"Red Mask.png")), red_mask)
        cv2.imwrite(os.path.join(save_base_path,model_num,(name+'_'+'_'+img_num+"Filled Ellipses.png")), filled)
        cv2.imwrite(os.path.join(save_base_path,model_num,(name+'_'+'_'+img_num+"Shoe Mask.png")), shoe_mask_image)
        cv2.imwrite(os.path.join(save_base_path,model_num,(name+'_'+'_'+img_num+'overlap_mask.png')), intersection)
        cv2.waitKey(0)
        
    df.to_excel(os.path.join(save_base_path,model_num,(name+'.xlsx')),index=False)

cv2.destroyAllWindows()


In [ ]:
img_array = np.fromfile('YOUR PATH', np.uint8)
image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
height, width, _ = image.shape
if height != 4000 and width != 2250:
    top_crop = 260
    bottom_crop = 260
    cropped_img = image[top_crop:height - bottom_crop, :]

    # 리사이즈 (4000x2250)
    image = cv2.resize(cropped_img, (4000, 2250), interpolation=cv2.INTER_LANCZOS4)
# 결과 출력
cv2.imshow("Original Image", image)
cv2.waitKey(0)
        

cv2.destroyAllWindows()
